In [ ]:
### Notebook for plotting graphics from polygons

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.geometry import MultiPolygon
from bs4 import BeautifulSoup


In [ ]:
#### Load data from kml file exported by Google Earth

file_path = ('data/kml/ballparks.kml')

with open(file_path) as file:

    xml_data = file.read()



# Initialize soup variables for parsing file
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
# layers = soup.Document.Folder.Folder
# polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml
df = pd.DataFrame(columns=['field', 'foul', 'fop'])


## Loop through the folders and extract the data
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Cleaning
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 

## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop any rows with empty fields
df = df[(df != 0).all(1)]

# ## remove any numberic characters and . from field names
# # There shouldn't be any anymore because of the validation I did before exporting kml
# df['field'] = df['field'].str.replace(r'\d+', '')

## Parse field names to get level column using regex
import re
re_mlb = re.compile(r'mlb', re.IGNORECASE)
re_pro = re.compile(r'pro', re.IGNORECASE)
re_college = re.compile(r'college', re.IGNORECASE)
re_youth = re.compile(r'youth', re.IGNORECASE)
re_muni = re.compile(r'muni', re.IGNORECASE)

df['level'] = df['field'].apply(lambda x: 'mlb' 
        if re_mlb.search(x) else 'pro' 
        if re_pro.search(x) else 'college' 
        if re_college.search(x) else 'youth' 
        if re_youth.search(x) else 'muni' 
        if re_muni.search(x) else 'high_school')

# clean up the field names
# remove the level from the field name
df['field'] = df['field'].str.replace(r'MLB', '')
df['field'] = df['field'].str.replace(r'pro', '')
df['field'] = df['field'].str.replace(r'college', '')
df['field'] = df['field'].str.replace(r'High School', 'HS')
# remove - from end of field name
df['field'] = df['field'].str.replace(r'- $', '')

## Output test csv
# df.to_csv('TEMP/level2_tost.csv', index=False)

df['foul_poly'] = df['foul'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})
df['fop_poly'] = df['fop'].apply(lambda x: {'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]})

In [ ]:
############ TRANSFORM ###############


# get home plate coords for each foul and fop
## this will be the first coordinate in the list of coordinates

df['foul_hp'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[0:2])))
df['fop_hp'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[0:2])))

# FLIP THE COORDINATES
df['foul_hp'] = df['foul_hp'].apply(lambda x: (x[1], x[0]))
df['fop_hp'] = df['fop_hp'].apply(lambda x: (x[1], x[0]))

## Drop the extra zeros from the coordinates in the foul and fop columns
df['foul'] = df['foul'].apply(lambda x: x.replace(',0', ','))
df['fop'] = df['fop'].apply(lambda x: x.replace(',0', ','))

## get the second coordinate in the list of coordinates
df['foul_2nd'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[3:5])))
df['fop_2nd'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[3:5])))

# Get the last coordinates in the list of coordinates
df['foul_last'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[-3:-1])))
df['fop_last'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[-3:-1])))
# Filp the coordinates
df['foul_last'] = df['foul_last'].apply(lambda x: (x[1], x[0]))
df['fop_last'] = df['fop_last'].apply(lambda x: (x[1], x[0]))

df['foul_2nd_last'] = df['foul'].apply(lambda x: tuple(map(float, x.split(',')[-6:-4])))
df['fop_2nd_last'] = df['fop'].apply(lambda x: tuple(map(float, x.split(',')[-6:-4])))

In [ ]:
### Fork To Output file with combined polygon
## Added 4-12-23

import shapely.geometry as geom
import shapely.ops as ops
from shapely import affinity

df['poly'] = df.apply(lambda x: ops.unary_union([geom.shape(x['foul_poly']), geom.shape(x['fop_poly'])]), axis=1)


In [ ]:
df.head()

In [ ]:
## Only Save the columns we need

new_df = df[['field', 'level', 'fop_hp', 'poly']]

In [ ]:
new_df.head()

# ## Output test csv
# new_df.to_csv('data/360_test/TEMP_plate_fullpoly.csv', index=False)


In [1]:
import pandas as pd
import re

# Define the extract_lat_lon function
def extract_lat_lon(poly_str):
    coords_str = re.findall(r'(-?\d+\.\d+) (-?\d+\.\d+)', poly_str)
    return [(float(lat), float(lon)) for lon, lat in coords_str]

# Read the CSV data
url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/TEMP_plate_fullpoly.csv"
data = pd.read_csv(url)

# Iterate through the rows and apply the function
data['clean_coords'] = data['poly'].apply(extract_lat_lon)

# Print the DataFrame with the new column
# print(data)


In [2]:
data.head()

## Drop unneeded columns
data = data.drop(['poly'], axis=1)

# rename columns
data = data.rename(columns={'fop_hp': 'home_plate', 'clean_coords': 'polygon'})

data.head()


,field,level,home_plate,polygon
0,Frankfort-Elberta Area HS,high_school,"(44.6344082, -86.2258505)","[(44.6336304, -86.226006), (44.6336295, -86.22..."
1,Ironwood HS,high_school,"(46.457154, -90.1514578)","[(46.4565313, -90.1506035), (46.456498, -90.15..."
2,Oakland A's - mlb,mlb,"(37.7513245, -122.2010782)","[(37.7511479, -122.1999704), (37.7511262, -122..."
3,Citi Field - NY Mets - mlb,mlb,"(40.7565867, -73.8460277)","[(40.7570454, -73.8450072), (40.7568808, -73.8..."
4,Citizens Bank Park - Philadelphia Phillies - mlb,mlb,"(39.9055805, -75.1665943)","[(39.9060907, -75.1656247), (39.9060753, -75.1..."


In [ ]:
#### Merge the foul_poly and fop_poly columns into a single polygon column

import shapely.geometry as geom
import shapely.ops as ops
from shapely import affinity

df['poly'] = df.apply(lambda x: ops.unary_union([geom.shape(x['foul_poly']), geom.shape(x['fop_poly'])]), axis=1)

### Project the polygon to a projected coordinate system and rotate it to be horizontal
import pyproj
from shapely.ops import transform

# project to a projected coordinate system and save to a column
project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
df['poly_proj'] = df['poly'].apply(lambda x: transform(project, x))

# # find the difference between the foul_angle and 135

# df['angle_diff'] = df['foul_angle'].apply(lambda x: abs(x - 135))

# #### MOVED TO NEXT BLOCK TOT ALTER AND TEST
# # rotate the polygon to be horizontal and save to a column
# df['poly_rot'] = df.apply(lambda x: affinity.rotate(x['poly_proj'], x['angle_diff'], origin=x['foul_hp']), axis=1)

In [ ]:
######### 12-30-22######

## I think this simple method might give me the adjustment angle I need to find


angle_list = []
delta_list = []

for i in range(len(df)):
      try:
            a = np.array(df['fop_hp'][i])
            b = np.array(df['fop_last'][i])
            c = np.array(df['fop_2nd'][i])

            ba = a - b
            bc = c - b

            s = np.arctan2(*ba)
            if s < 0:
                  s += 2*np.pi

            e = np.arctan2(*bc)
            if e < 0:
                  e += 2*np.pi

            delta = e - s
            angle = np.degrees(delta)
            
            angle_list.append(angle)
            delta_list.append(delta)
            
            
      except:
            delta_list.append(np.nan)
            angle_list.append(np.nan)

      

df['delta'] = delta_list
df['angle'] = angle_list
df.info()

### Cleaning
## drop rows with null values

df = df.dropna()

df.info()

In [ ]:
df['angle'].hist(bins=100)

df['angle'].describe()

In [ ]:
########### 12/30/22 ##########
# Taken from above

#### Merge the foul_poly and fop_poly columns into a single polygon column

import shapely.geometry as geom
import shapely.ops as ops
from shapely import affinity

df['poly'] = df.apply(lambda x: ops.unary_union([geom.shape(x['foul_poly']), geom.shape(x['fop_poly'])]), axis=1)

### Project the polygon to a projected coordinate system and rotate it to be horizontal
import pyproj
from shapely.ops import transform

# project to a projected coordinate system and save to a column
project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True).transform
df['poly_proj'] = df['poly'].apply(lambda x: transform(project, x))

## make a shapely point out of the foul_hp and save to a column
df['foul_hp'] = df['foul_hp'].apply(lambda x: geom.Point(x))
df['hp_proj'] = df['foul_hp'].apply(lambda x: transform(project, x))

# # rotate the polygon to be horizontal and save to a column and skip null values

## seperate dataframe into two dataframes, one with angles above 45 and one with angles below 45
df_above = df[df['angle'] > 45]
df_below = df[df['angle'] < 45]


for i in range(len(df_above)):
    try:    
        df_above['poly_rot'] = df.apply(lambda x: affinity.rotate(x['poly_proj'], -abs(x['angle']-45), origin=x['foul_hp']), axis=1)
    except:
        pass

for i in range(len(df_below)):
    try:
        df_below['poly_rot'] = df.apply(lambda x: affinity.rotate(x['poly_proj'], (45 - x['angle']), origin=x['foul_hp']), axis=1)
    except:
        pass
    
## Recombine the daaframes

df = pd.concat([df_above, df_below])

In [ ]:
#### 12-30-22 ####
# Test some plot and take better look at rotate function

import matplotlib.pyplot as plt
import descartes

i = 3

shape = geom.shape(df['poly_proj'][i])
shape_rot = geom.shape(df['poly_rot'][i])


x, y = shape.exterior.xy
x2, y2 = shape_rot.exterior.xy

### Create a point to plot from the home plate coordinates
hp = df['hp_proj'][i]


## Rotated polygon
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(x2, y2, hp, color='red', alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)

ax.set_aspect('equal')
ax.set_title(df['field'][i] + ' - Rotated')
ax.set_ylabel(df['angle'][i])
ax.set_xlabel(df['angle'][i]-45)

plt.show()

## Original polygon
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(x, y, color='#6699cc', alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)

ax.set_aspect('equal')
ax.set_title(df['field'][i] + ' - Original')
ax.set_ylabel(df['angle'][i])
ax.set_xlabel(df['angle'][i]-45)

plt.show()

In [ ]:
### Output plots to a folder

import os

import matplotlib.style as style
style.use('tableau-colorblind10')

plot_dir = 'TEMP\plots'


##Output the rotated polygons with the hp point also in the figure
# set the background to transparent

### Plot the rot_poly column
import matplotlib.pyplot as plt
import descartes
failed = []
# i = 160

for i in range(0, len(df)):
        try:
                shape = geom.shape(df['poly_rot'][i])
                x, y = shape.exterior.xy

                # point = geom.point(df['foul_hp'][i])
                


                fig, ax = plt.subplots(figsize=(10,10))
                

                ax.plot(x, y, alpha=0.7,
                        linewidth=3, solid_capstyle='round', zorder=2)
                # ax.plot(point, color='red', alpha=0.7)
                ax.set_aspect('auto')
                ax.set_title(df['field'][i])
                ax.set_ylabel(df['angle'][i])
                ax.set_xlabel(f'index: {i}')
                
                ### Save the figure to a folder
                fig.savefig(os.path.join(plot_dir, df['field'][i] + '.png'), transparent=True)

                plt.close()

                
                
        except:
                failed.append(i)
                pass
        

print(failed)
                

        
print('failed', failed)




